## Colab 환경에서 하는 것을 권장합니다.

# 사전 준비

In [123]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [125]:
cd drive/MyDrive/Data

[Errno 2] No such file or directory: 'drive/MyDrive/Data'
/content/drive/MyDrive


In [ ]:
#konlpy 설치 (mecab 제외). 3~40초 정도 소요
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

In [126]:
# java 경로 설정
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [127]:
# 필요 라이브러리 불러오기
import os, json
from tqdm.notebook import tqdm

from konlpy.tag import Kkma, Komoran, Hannanum, Okt, Mecab

from gensim.models import Word2Vec

import plotly
import plotly.graph_objs as go
import matplotlib.pyplot as plt
from sklearn.datasets import make_swiss_roll
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [129]:
os.getcwd()

'/content/drive/MyDrive'

In [130]:
# data 불러오기
data_dir = os.path.join(os.getcwd(), 'Data' ,'data')

corpus = []

for i in tqdm(os.listdir(data_dir)) :

    with open(os.path.join(data_dir, i), 'r') as f :

        json_data = json.load(f)
    
    corpus.append(json_data['sentences'])

corpus = sum(corpus, [])

  0%|          | 0/50 [00:00<?, ?it/s]

데이터를 확인하고 어떻게 전처리를 하고 토큰화를 해야 할 지 고민해보세요!

In [ ]:
corpus[:50]

['16일 화성 궤도에 진입한 유럽우주국(ESA) 무인 화성탐사선 ‘엑소마스(EXoMars)’ 상상도',
 'ESA 제공',
 '러시아와 유럽이 공동으로 진행하는 화성 생명체 탐사 프로젝트 ‘엑소마스(EXoMars)’의 화성 탐사 착륙선 ‘스키아파렐리’(Schiaparelli)가 화성에 터치다운을 앞두고 교신이 두절됐다',
 '유럽우주국(ESA)은 그리니치 표준시(GMT)로 19일 오후 2시 48분께 화성에 착륙할 예정이던 화성 탐사 착륙선 ‘스키아파렐리’와 ESA 간의 교신이 끊겨 안착 여부가 불투명하다고 이날 발표했다',
 'ESA는 스키아파렐리가 화성 탐사선을 구성한 궤도선인 ‘가스추적궤도선’(TGO)에서 성공적으로 분리된 후 화성 착륙을 앞두고 특정 지점에서 교신이 끊겼다고 밝혔다',
 'TGO는 생명체가 화성에 살아있음을 보여주는 메탄가스를 탐지하는 역할을 한다',
 'ESA 과학자들은 “좋은 징후가 아니다”라면서도 “실패라고 판단하기엔 너무 이르다”며 신중한 태도를 보이고 있다',
 '화성 생명체 탐사를 위한 무인 탐사선 ‘엑소마스’(ExoMars) 프로젝트의 일환인 스키아파렐리는 올해 3월 14일 카자흐스탄 바이코누르 우주기지에서 프로톤 로켓에 실려 발사됐다',
 '약 7개월간 5억 km가량을 날아 화성 궤도에 들어섰다',
 '스키아파렐리는 화성 표면의 온도와 습도, 밀도, 전기적 성질 등의 자료와 사진 이미지를 보낼 예정이었다',
 'ESA는 러시아 연방우주공사(로스코스모스)와 협력, 2020년 탐사선과 탐사 로봇을 화성에 보내 생명체의 흔적을 본격적으로 탐사하는 것을 목표로 하고 있다',
 '한편 지금까지 화성 탐사 착륙선을 화성 표면에 착륙시킨 나라는 미국과 러시아(구소련 포함) 뿐이다',
 '‘내장형 심장감시장치 장착 테이저’',
 '테이저 때문에 죽는 사람은 의외로 많다',
 '‘워싱턴 포스트’ 지의 2015년 보도에 따르면, 경찰에게 테이저로 사격을 당한 후 구속 중 사망하는 사람이 1주에 1명 꼴로 발생한다고 한다',
 '전도성 전기 

# 1번

자신만의 CustomTokenization을 만들어보세요! 세션에서 말씀 드렸듯이 정답이 있는 것이 아닙니다. 

제가 세션에서 설명했던 것 처럼 pos를 이용하여 품사를 구하고 품사를 통해 stop words를 지정해서 제거해 줄 수 있고, 따로 stopw words를 지정해서 의미 없다고 생각하는 단어들을 제거해도 상관 없습니다!

물론 Okt말고 다른 Tokenizer를 사용해도 괜찮습니다. 

다양한 방법으로 전처리를 진행해보고 왜 그런 전처리와 tokenizing 방법을 사용했는지 설명해주세요!

[여러분의 설명]

In [131]:
kkm = Kkma()
kom = Komoran()
han = Hannanum()
okt = Okt()

In [ ]:
ex1 = corpus[2:3][0]

ex1

'러시아와 유럽이 공동으로 진행하는 화성 생명체 탐사 프로젝트 ‘엑소마스(EXoMars)’의 화성 탐사 착륙선 ‘스키아파렐리’(Schiaparelli)가 화성에 터치다운을 앞두고 교신이 두절됐다'

In [ ]:
kom.morphs(ex1)

['러시아와 유럽',
 '이',
 '공동',
 '으로',
 '진행',
 '하',
 '는',
 '화성',
 '생명체',
 '탐사',
 '프로젝트',
 '‘엑소마스(EXoMars)’의',
 '화성 탐사',
 '착륙선',
 '‘',
 '스키아파렐리',
 '’',
 '(',
 'Schiaparelli',
 ')',
 '가',
 '화성',
 '에',
 '터치',
 '다운',
 '을',
 '앞두',
 '고',
 '교신',
 '이',
 '두절',
 '되',
 '었',
 '다']

In [ ]:
kom.pos(ex1)

[('러시아와 유럽', 'NNP'),
 ('이', 'JKS'),
 ('공동', 'NNG'),
 ('으로', 'JKB'),
 ('진행', 'NNG'),
 ('하', 'XSV'),
 ('는', 'ETM'),
 ('화성', 'NNP'),
 ('생명체', 'NNG'),
 ('탐사', 'NNG'),
 ('프로젝트', 'NNP'),
 ('‘엑소마스(EXoMars)’의', 'NA'),
 ('화성 탐사', 'NNP'),
 ('착륙선', 'NNP'),
 ('‘', 'SS'),
 ('스키아파렐리', 'NNP'),
 ('’', 'SS'),
 ('(', 'SS'),
 ('Schiaparelli', 'SL'),
 (')', 'SS'),
 ('가', 'JKS'),
 ('화성', 'NNP'),
 ('에', 'JKB'),
 ('터치', 'NNP'),
 ('다운', 'NNP'),
 ('을', 'JKO'),
 ('앞두', 'VV'),
 ('고', 'EC'),
 ('교신', 'NNG'),
 ('이', 'JKS'),
 ('두절', 'NNG'),
 ('되', 'XSV'),
 ('었', 'EP'),
 ('다', 'EC')]

In [132]:
# 자신만의 tokenization, preprocess 규칙을 만들고 그 규칙에 맞는 CustomTokenizer를 만들어보세요.
# 한 문장을 input으로 받아 token이 들어있는 list를 반환하는 함수를 구현해보세요.
def CustomTokenization(sentence) :

  token = []

  stop_pos = ['JKS' , 'JKC' , 'JKG' , 'JKO' , 'JKB' , 'JKV' , 'JKQ' , 'JC' , 'SS']

  for i, j in kom.pos(sentence) :

    if j in stop_pos :

      continue
    
    token.append(i)

  return token

In [ ]:
CustomTokenization(ex1)

['러시아와 유럽',
 '공동',
 '진행',
 '하',
 '는',
 '화성',
 '생명체',
 '탐사',
 '프로젝트',
 '‘엑소마스(EXoMars)’의',
 '화성 탐사',
 '착륙선',
 '스키아파렐리',
 'Schiaparelli',
 '화성',
 '터치',
 '다운',
 '앞두',
 '고',
 '교신',
 '두절',
 '되',
 '었',
 '다']

In [133]:
# corpus_tokenized 안에 CustomTokenization의 return값이 들어 있는 2중 list를 만들어 보세요
corpus_tokenized = []

for i in tqdm(corpus) :
  
    corpus_tokenized.append(CustomTokenization(i))

  0%|          | 0/1020 [00:00<?, ?it/s]

# 2번

이제 word2vec 모델을 학습시켜 봅시다!

1. 사전학습 없이 모델 학습
2. 사전학습 된 모델을 사용하여 학습

두가지를 진행해보세요

hyperparameter는 본인이 하고 싶으신 것으로 하면 됩니다. 단, 사전학습 된 모델을 사용할 때는 size = 200으로 맞추어 주세요!

hyperparameter 설정 기준을 설명해주세요!

[여러분의 설명]

In [134]:
# 사전학습 없이 모델 학습
# 모델 생성
model_no_pretrained = Word2Vec(min_count = 3 , window = 7)

# 모델 vocabulary 추가
model_no_pretrained.build_vocab(corpus_tokenized)

total_examples = model_no_pretrained.corpus_count
print('현재 vocabulary size(V) :',len(model_no_pretrained.wv.vocab))

현재 vocabulary size(V) : 1387


In [135]:
# 모델 학습
model_no_pretrained.train(corpus_tokenized, total_examples = total_examples, epochs = 100)

(1528055, 2486400)

In [ ]:
model_no_pretrained.wv.vocab.keys()

dict_keys(['16', '일', '화성', '궤도', '진입', '하', 'ㄴ', '유럽', 'ESA', '무인', '탐사선', '도', '제공', '공동', '진행', '는', '생명체', '탐사', '프로젝트', '화성 탐사', '착륙선', '스키아파렐리', '고', '교신', '되', '었', '다', '은', '분', '착륙', 'ㄹ', '예정', '이', '던', '간', '어', '다고', '이날', '발표', '았', '선', '구성', '가스', '추적', '성공', '적', '분리', '후', '특정', '밝히', '살', '아', '있', '음', '보이', '주', '탐지', '역할', 'ㄴ다', '과학자', '들', '좋', '아니', '실패', '라고', '판단', '기', '이르', '며', '위하', '일환', '올해', '로켓', '실리', '발사', '약', '7', '개월', '5', '억', 'km', '가량', '표면', ',', '밀도', '전기', '성질', '등', '자료', '사진', '이미지', '보내', '러시아', '우주', '협력', '2020', '년', '로봇', '본격', '것', '목표', '한편', '지금', '까지', '시키', '나라', '미국', '포함', '뿐', '내', '심장', '감시', '장치', '장착', '테이저', '때문', '죽', '사람', '의외로', '많', '2015', '보도', '따르', '면', '경찰', '중', '사망', '1', '명', '발생', 'ㄴ다고', '인하', '대부분', '의하', '인해', '생기', '그러나', '일으키', '수', '나타나', '현재', '여러', '연구자', 'EKG', '기기', '추가', '피의자', '심', '부담', 'ㅁ', '막', '을', '생각', '웨이', '크', '보즈', '멀', '지나', '동료', '함께', '안전', '성', '검증', '그', '.', '75', '%', '없', '거나', '교

In [ ]:
# 모델이 잘 학습 되었는지 자유롭게 확인해보세요!
# .wv.most_similar() 등 사용

model_no_pretrained.wv.most_similar('화성')

[('탐사', 0.8432472944259644),
 ('유인', 0.8425458669662476),
 ('생명체', 0.8315670490264893),
 ('화성 탐사', 0.827475368976593),
 ('스키아파렐리', 0.8220003843307495),
 ('착륙선', 0.7828906774520874),
 ('보내', 0.7816659212112427),
 ('착륙', 0.7699587941169739),
 ('스페이스X', 0.7661793231964111),
 ('\u3000', 0.7579215168952942)]

In [138]:
%cd /content/drive/MyDrive/Data

/content/drive/MyDrive/Data


In [139]:
# 사전학습 된 모델을 사용하여 학습
#사전학습 된 모델 불러오기
pretrained = Word2Vec.load('ko.bin')

print('Pretrained model의 단어 수(V) :', len(pretrained.wv.vocab))
print('Pretrained model의 embedding vector의 size(N) :', len(pretrained.wv['강아지']))

# 모델 생성
model_pretrained = Word2Vec(min_count = 3 , window = 7)

# 모델 vocabulary 추가
model_pretrained.build_vocab(corpus_tokenized)

total_examples = model_pretrained.corpus_count
print('현재 vocabulary size(V) :',len(model_pretrained.wv.vocab))

FileNotFoundError: ignored

In [ ]:
#pretrained model의 학습되어 있는 vocabulary list 불러오기
model_pretrained.build_vocab([list(pretrained.wv.vocab.keys())], update = True)
print('현재 vocabulary size(V) :', len(model_pretrained.wv.vocab))

#embedding vector 까지 불러오기. lockf = 0.0 이면 불러온 벡터에 대해서는 재학습이 불가함. 1.0이면 재학습 가능.
model_pretrained.intersect_word2vec_format('ko.bin.gz', binary = False, lockf = 1.0)

In [ ]:
# 모델 학습
model_pretrained.train(corpus_tokenized, total_examples = total_examples, epochs = 100)

In [ ]:
# 모델이 잘 학습 되었는지 자유롭게 확인해보세요!
# .wv.most_similar() 등 사용

model_pretrained.wv.most_similar('화성')

# 3번

사전 학습 없이 모델을 학습해도 성능이 괜찮나요?   
사전 학습 없이 학습한 모델과 사전 학습이 된 모델을 사용한 모델 간의 성능 차이가 있었나요? 차이가 있었다면 어떤 부분에서 차이가 나고 왜 그런 차이가 났는지 생각해보고 설명해주세요!

[여러분의 설명]

# 4번

이제 학습된 모델을 통해 나온 embedding 결과를 2차원, 3차원에 embedding 시켜 봅시다.    
사전학습된 데이터를 사용하면 너무 단어 수가 많아지기 때문에 우리가 처음에 가지고 있던 corpus_tokenized 데이터만 사용하겠습니다.   
PCA, t-SNE를 사용해서 시각화하고 그 두가지 방법의 결과를 비교해보세요!   

추가적으로 무언가 재미있는 시각화 결과 (ex. 이거랑 저거랑 비슷한 위치에 임베딩 됐어요!) 가 있다면 추가적으로 남겨주세요!

[여러분의 설명]

In [ ]:
# 시각화할 단어 리스트 생성
unique_text = list(set(sum(corpus_tokenized, [])))
len(unique_text)

3991

In [ ]:
# embedding 결과 가져오기
word, vector = [], []
for i in unique_text :

    word.append(i)
    
    vector.append(model_no_pretrained.wv[i] or model_pretrained.wv[i])

ValueError: ignored

In [ ]:
# 시각화해보세요!
import plotly
import plotly.graph_objs as go
from sklearn.datasets import make_swiss_roll
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [ ]:
from tqdm import tqdm

vocab = []
vector = []

for i in tqdm(model_no_pretrained.wv.vocab.keys()):
    
    vocab.append(i)
    vector.append(model_no_pretrained.wv[i])

100%|██████████| 1387/1387 [00:00<00:00, 144415.75it/s]


In [ ]:
three_dim = PCA(random_state = 0).fit_transform(vector)[:,:3]

In [ ]:
data = go.Scatter3d(
    x = three_dim[:, 0],
    y = three_dim[:, 1],
    z = three_dim[:, 2],
    text = vocab,
    marker = dict(
        size = 3,
        colorscale = 'Jet',
        opacity = 0.8
    )
)

layout = go.Layout(
    autosize = False,
    width = 1000,
    height = 1000,
    margin = go.Margin(
        l = 50,
        r = 50,
        b = 100,
        t = 100,
        pad = 4
    )
)

fig = go.Figure(data = [data], layout = layout)
fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




# 5번

t-SNE에는 perplexity라는 hyperparameter가 존재했는데요. 5~50 사이의 값에서 robust 한 결과를 보인다고는 하지만 perplexity가 달라지면서 embedding 결과가 미묘하지만 크게 바뀌기도 합니다. perplexity와 n_iter 조합을 달리해보면서 perplexity의 변화가 embedding결과에 어떤 영향을 미치는지 살펴보세요!

In [ ]:
# t-SNE
tsne = TSNE(n_components = 2, random_state = 0, perplexity = 10, learning_rate = 0.05, init = 'random', n_iter = 1000).fit_transform(vector)[:, :2]

print(tsne.shape)

tsne

(1387, 2)


array([[-2.1662648 ,  2.2675354 ],
       [-2.8723416 ,  2.8740697 ],
       [-3.7662964 ,  1.7887622 ],
       ...,
       [-2.0455945 , -1.9908266 ],
       [ 0.31334388,  2.29949   ],
       [ 0.54570407, -0.5880154 ]], dtype=float32)

In [ ]:
# 시각화
data = go.Scatter(
    x=tsne[:,0],
    y=tsne[:,1],
    text = vocab,
    mode='markers',
    marker=dict(
        size=5,
        color=color,
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




In [ ]:
# t-SNE
tsne = TSNE(n_components = 2, random_state = 0, perplexity = 25, learning_rate = 0.05, init = 'random', n_iter = 1000).fit_transform(vector)[:, :2]

print(tsne.shape)

tsne

(1387, 2)


array([[-1.428823 ,  2.116432 ],
       [-2.7276404,  2.465457 ],
       [-2.4869545,  1.8664862],
       ...,
       [-1.9207036, -1.7471449],
       [ 0.6678428,  1.4768777],
       [ 0.6289203, -0.946973 ]], dtype=float32)

In [ ]:
# 시각화
data = go.Scatter(
    x=tsne[:,0],
    y=tsne[:,1],
    text = vocab,
    mode='markers',
    marker=dict(
        size=5,
        color=color,
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




# 수고하셨습니다!